In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm_notebook, tqdm
import jieba

In [2]:
# 读入数据，分析不同数据的情况
# /home/zqxie/project/SimCSE-Chinese-Pytorch-main/datasets/tianchi_data/data_check.py
corpus_data = pd.read_csv( "./datasets/tianchi_data/corpus.tsv", sep="\t", names=["doc", "title"]) # doc文档
dev_data = pd.read_csv("./datasets/tianchi_data/dev.query.txt", sep="\t", names=["query", "title"]) # 测试集 que文档
train_data = pd.read_csv("./datasets/tianchi_data/train.query.txt", sep="\t", names=["query", "title"]) # 训练集que文档
qrels = pd.read_csv("./datasets/tianchi_data/qrels.train.tsv", sep="\t", names=["query", "doc"]) # 一一对应还是一对多？
# 训练集中唯一的query: 10w条
# doc: 10w,query和doc是一一对应的

In [3]:
corpus_data = corpus_data.set_index("doc")
dev_data = dev_data.set_index("query")
train_data = train_data.set_index("query")
qrels = qrels.set_index("query")

In [4]:
# 计算TF-IDF 词频 将高频词送入word2vec训练
def time_out(x):
    return list(jieba.cut(x))
# 并行处理
from joblib import Parallel, delayed
corpus = Parallel(n_jobs=4)(delayed(time_out)(i) for i in corpus_data['title'])
train = Parallel(n_jobs=4)(delayed(time_out)(i) for i in train_data['title'])
test = Parallel(n_jobs=4)(delayed(time_out)(i) for i in dev_data['title'])

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
idf = TfidfVectorizer(analyzer=lambda x: x)
idf.fit(train+ corpus + test)

TfidfVectorizer(analyzer=<function <lambda> at 0x7fcbf0296488>, binary=False,
                decode_error='strict', dtype=<class 'numpy.float64'>,
                encoding='utf-8', input='content', lowercase=True, max_df=1.0,
                max_features=None, min_df=1, ngram_range=(1, 1), norm='l2',
                preprocessor=None, smooth_idf=True, stop_words=None,
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [9]:
idf_corpus = idf.transform(corpus)

In [51]:
import random
from snownlp import SnowNLP

IDF_THREAD = 7
feature_names = np.array(idf.get_feature_names())
corpus_final = []
rate = 0.6
with open('./data/unsupervised_data_random_cut_random_rate.txt', 'w') as up:
    for i in tqdm(range(1,len(corpus)+1)):
        doc = corpus_data.loc[i]['title']
        doc = doc.strip()
        # 转化为简体
        doc = SnowNLP(doc).han
        len_ = len(doc)
        
        random_rate = random.uniform(0.5, 0.8)
        begin_idx = random.randint(0,len_-int(len_*random_rate))
        doc_cur = doc[begin_idx:begin_idx+int(len_*random_rate)]
#         print(doc, '\n', doc_cur)
        cur = '{}\t{}'.format(doc_cur, doc)
        if len(cur.strip().split("\t"))<2:
            continue
        else:
            up.write('{}\n'.format(cur))
#         if i>100:
#             break

100%|██████████| 1001500/1001500 [04:29<00:00, 3720.26it/s]


In [49]:
random.uniform(0.5, 0.8)

0.6078519946860589

In [44]:
with open('./data/unsupervised_data.txt','r') as up:
    lines = up.readlines()
    for line in lines:
        cur = line.strip().split("\t")
        if len(cur)<2:
            print(cur)
            print(line)
            break

In [36]:
doc = '      	番茄家代号链接      进群请看图进哦，～'
doc = doc.strip()
len_ = len(doc)
begin_idx = random.randint(0,len_-int(len_*rate))
doc_cur = doc[begin_idx:begin_idx+int(len_*rate)]

print(doc)
print(doc_cur)

番茄家代号链接      进群请看图进哦，～
代号链接      进群请
